# Semi-Supervised Learning para Classificação de Sintomas Médicos

Este notebook demonstra como implementar aprendizado semi-supervisionado para melhorar a classificação de sintomas médicos em diagnósticos, usando o dataset "gretelai/symptom_to_diagnosis" do HuggingFace.

Implementamos a técnica de pseudo-rotulagem (pseudo-labeling) para aproveitar dados não rotulados e melhorar o desempenho do modelo.

## 1. Instalação de Dependências

In [ ]:
# Instalar dependências necessárias
!pip install torch transformers datasets scikit-learn pandas numpy

## 2. Importação de Bibliotecas

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, ConcatDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
import random
import numpy as np
import os
import argparse
from dataclasses import dataclass
from typing import Dict, List, Union, Optional

# Verificar disponibilidade de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

## 3. Funções Utilitárias

In [ ]:
def set_seed(seed=42):
    """
    Configura sementes aleatórias para reprodutibilidade.
    
    Args:
        seed: Valor da semente
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def get_device():
    """
    Determina o dispositivo disponível (GPU ou CPU).
    
    Returns:
        device: Dispositivo PyTorch
    """
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def print_dataset_examples(dataset, num_examples=3):
    """
    Imprime exemplos do dataset para inspeção.
    
    Args:
        dataset: Dataset a ser inspecionado
        num_examples: Número de exemplos a serem exibidos
    """
    print("\nExemplos do dataset:")
    for i in range(min(num_examples, len(dataset['train']))):
        print(f"Exemplo {i+1}:")
        print(f"Sintomas: {dataset['train'][i]['input_text']}")
        print(f"Diagnóstico: {dataset['train'][i]['output_text']}")
        print()

def print_comparison_results(eval_results, final_eval_results):
    """
    Imprime uma comparação dos resultados antes e depois do treinamento semi-supervisionado.
    
    Args:
        eval_results: Resultados da avaliação após treinamento supervisionado
        final_eval_results: Resultados da avaliação após treinamento semi-supervisionado
    """
    print("\nComparação dos resultados:")
    print(f"Acurácia (supervisionado): {eval_results['eval_accuracy']:.4f}")
    print(f"Acurácia (semi-supervisionado): {final_eval_results['eval_accuracy']:.4f}")
    print(f"Melhoria na acurácia: {final_eval_results['eval_accuracy'] - eval_results['eval_accuracy']:.4f}")
    
    print(f"F1-score (supervisionado): {eval_results['eval_f1']:.4f}")
    print(f"F1-score (semi-supervisionado): {final_eval_results['eval_f1']:.4f}")
    print(f"Melhoria no F1-score: {final_eval_results['eval_f1'] - eval_results['eval_f1']:.4f}")

## 4. Funções de Processamento de Dados

In [ ]:
def load_symptom_diagnosis_dataset(max_samples=None):
    """
    Carrega o dataset de sintomas e diagnósticos.
    
    Args:
        max_samples: Se especificado, limita o número de amostras para teste rápido
        
    Returns:
        dataset: Dataset carregado
    """
    dataset = load_dataset("gretelai/symptom_to_diagnosis")
    
    if max_samples and len(dataset['train']) > max_samples:
        dataset['train'] = dataset['train'].select(range(max_samples))
    
    return dataset

def prepare_labels(dataset):
    """
    Prepara os rótulos e retorna um encoder para os diagnósticos.
    
    Args:
        dataset: Dataset contendo rótulos em 'output_text'
        
    Returns:
        label_encoder: O encoder treinado nos rótulos
        num_labels: Número de classes únicas
    """
    label_encoder = LabelEncoder()
    all_labels = dataset['train']['output_text']
    label_encoder.fit(all_labels)
    num_labels = len(label_encoder.classes_)
    
    return label_encoder, num_labels

def tokenize_dataset(dataset, tokenizer):
    """
    Tokeniza o dataset usando o tokenizer fornecido.
    
    Args:
        dataset: Dataset a ser tokenizado
        tokenizer: Tokenizer a ser usado
        
    Returns:
        tokenized_dataset: Dataset tokenizado
    """
    def tokenize_function(examples):
        return tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=128)
    
    return dataset.map(tokenize_function, batched=True)

def create_semi_supervised_split(tokenized_dataset, dataset, label_encoder, tokenizer, labeled_ratio=0.2):
    """
    Cria divisões de dados para aprendizado semi-supervisionado.
    
    Args:
        tokenized_dataset: Dataset tokenizado
        dataset: Dataset original com rótulos
        label_encoder: Encoder para os rótulos
        tokenizer: Tokenizer para processar os textos
        labeled_ratio: Fração de dados rotulados a serem usados
        
    Returns:
        labeled_dataset: TensorDataset com dados rotulados
        unlabeled_dataset: TensorDataset com dados não rotulados
        unlabeled_indices: Índices dos dados não rotulados
        labeled_indices: Índices dos dados rotulados
        unlabeled_tokenized: Tokens dos dados não rotulados
    """
    # Embaralhar os índices
    train_indices = list(range(len(tokenized_dataset["train"])))
    random.shuffle(train_indices)
    
    # Dividir em conjunto rotulado e não-rotulado
    num_labeled = int(labeled_ratio * len(train_indices))
    labeled_indices = train_indices[:num_labeled]
    unlabeled_indices = train_indices[num_labeled:]
    
    # Função para codificar os rótulos
    def encode_labels(labels):
        return label_encoder.transform(labels)
    
    # Criar conjuntos de dados rotulados
    labeled_texts = [tokenized_dataset["train"][i]["input_text"] for i in labeled_indices]
    labeled_tokenized = tokenizer(labeled_texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    labeled_labels = [dataset["train"][i]["output_text"] for i in labeled_indices]
    
    # Converter rótulos para long
    labeled_encoded_labels = torch.tensor(encode_labels(labeled_labels), dtype=torch.long)
    
    labeled_dataset = TensorDataset(
        labeled_tokenized["input_ids"],
        labeled_tokenized["attention_mask"],
        labeled_encoded_labels
    )
    
    # Criar conjuntos de dados não-rotulados
    unlabeled_texts = [tokenized_dataset["train"][i]["input_text"] for i in unlabeled_indices]
    unlabeled_tokenized = tokenizer(unlabeled_texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    
    unlabeled_dataset = TensorDataset(
        unlabeled_tokenized["input_ids"],
        unlabeled_tokenized["attention_mask"]
    )
    
    return labeled_dataset, unlabeled_dataset, unlabeled_indices, labeled_indices, unlabeled_tokenized

def split_train_val(dataset, val_ratio=0.1):
    """
    Divide um dataset em conjuntos de treino e validação.
    
    Args:
        dataset: Dataset a ser dividido
        val_ratio: Fração de dados para validação
        
    Returns:
        train_dataset, val_dataset: Datasets de treino e validação
    """
    val_size = int(len(dataset) * val_ratio)
    train_size = len(dataset) - val_size
    return random_split(dataset, [train_size, val_size])

## 5. Funções de Modelo

In [ ]:
def get_tokenizer(model_name="distilbert/distilbert-base-uncased"):
    """
    Obtém o tokenizer para o modelo especificado.
    
    Args:
        model_name: Nome do modelo pré-treinado
        
    Returns:
        tokenizer: Tokenizer configurado
    """
    return AutoTokenizer.from_pretrained(model_name)

def get_model(model_name="distilbert/distilbert-base-uncased", num_labels=22):
    """
    Cria e retorna o modelo de classificação.
    
    Args:
        model_name: Nome do modelo pré-treinado
        num_labels: Número de classes para classificação
        
    Returns:
        model: Modelo de classificação configurado
    """
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    return model

def compute_metrics(pred):
    """
    Calcula as métricas de avaliação para o modelo.
    
    Args:
        pred: Saída de predição do modelo
        
    Returns:
        metrics: Dicionário com métricas calculadas
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def generate_pseudo_labels(model, dataloader, device):
    """
    Gera pseudo-rótulos para dados não rotulados.
    
    Args:
        model: Modelo treinado
        dataloader: DataLoader com dados não rotulados
        device: Dispositivo (CPU ou GPU)
        
    Returns:
        pseudo_labels: Lista de pseudo-rótulos gerados
    """
    model.eval()
    pseudo_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            pseudo_labels.extend(predictions.cpu().numpy())
    
    return pseudo_labels

## 6. Funções de Treinamento

In [ ]:
@dataclass
class DataCollatorForTextClassification:
    """
    Collator personalizado para classificação de textos que lida com nossos TensorDatasets.
    """
    def __call__(self, features):
        """
        Agrupa elementos em um batch.
        
        Args:
            features: Lista de elementos do dataset
            
        Returns:
            Batch de dados formatado corretamente
        """
        # Cada feature é (input_ids, attention_mask, label) de um TensorDataset
        if isinstance(features[0], tuple):
            input_ids = torch.stack([f[0] for f in features])
            attention_mask = torch.stack([f[1] for f in features])
            
            if len(features[0]) > 2:  # Se tiver rótulos
                # Garantir que os rótulos sejam do tipo long
                labels = torch.stack([f[2] for f in features]).long()
                return {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "labels": labels
                }
            
            return {
                "input_ids": input_ids,
                "attention_mask": attention_mask,
            }
        
        # Fallback para o comportamento padrão
        batch = {k: torch.stack([f[k] for f in features]) for k in features[0].keys()}
        
        # Garantir que os rótulos sejam long se existirem
        if "labels" in batch:
            batch["labels"] = batch["labels"].long()
            
        return batch

def get_training_args(output_dir, num_train_epochs=3, batch_size=16, eval_batch_size=64):
    """
    Configura os argumentos de treinamento.
    
    Args:
        output_dir: Diretório para salvar resultados
        num_train_epochs: Número de épocas de treinamento
        batch_size: Tamanho do batch para treinamento
        eval_batch_size: Tamanho do batch para avaliação
        
    Returns:
        training_args: Argumentos de treinamento configurados
    """
    # Usar apenas argumentos essenciais básicos
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=eval_batch_size,
        weight_decay=0.01,
        logging_dir=f"{output_dir}/logs",
    )
    
    return training_args

def train_supervised_model(model, tokenizer, labeled_train_dataset, labeled_val_dataset, compute_metrics, output_dir="./results_supervised"):
    """
    Treina o modelo usando apenas dados rotulados (aprendizado supervisionado).
    
    Args:
        model: Modelo a ser treinado
        tokenizer: Tokenizer usado
        labeled_train_dataset: Dataset de treino rotulado
        labeled_val_dataset: Dataset de validação
        compute_metrics: Função para calcular métricas
        output_dir: Diretório para salvar resultados
        
    Returns:
        trainer: Objeto Trainer treinado
        eval_results: Resultados da avaliação
    """
    training_args = get_training_args(output_dir)
    data_collator = DataCollatorForTextClassification()
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=labeled_train_dataset,
        eval_dataset=labeled_val_dataset,
        compute_metrics=compute_metrics,
        data_collator=data_collator,  # Usar o collator personalizado
    )
    
    trainer.train()
    eval_results = trainer.evaluate()
    
    return trainer, eval_results

def train_semi_supervised_model(model, pseudo_labeled_dataset, labeled_train_dataset, labeled_val_dataset, compute_metrics, output_dir="./results_semi_supervised"):
    """
    Treina o modelo usando dados rotulados e pseudo-rotulados (aprendizado semi-supervisionado).
    
    Args:
        model: Modelo pré-treinado com dados rotulados
        pseudo_labeled_dataset: Dataset com pseudo-rótulos
        labeled_train_dataset: Dataset de treino rotulado
        labeled_val_dataset: Dataset de validação
        compute_metrics: Função para calcular métricas
        output_dir: Diretório para salvar resultados
        
    Returns:
        trainer: Objeto Trainer treinado
        eval_results: Resultados da avaliação
    """
    # Combinar dados rotulados com pseudo-rotulados
    combined_dataset = ConcatDataset([labeled_train_dataset, pseudo_labeled_dataset])
    
    # Configurar treinamento
    training_args = get_training_args(
        output_dir=output_dir,
        num_train_epochs=2  # Menos épocas para o treinamento semi-supervisionado
    )
    
    data_collator = DataCollatorForTextClassification()
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=combined_dataset,
        eval_dataset=labeled_val_dataset,
        compute_metrics=compute_metrics,
        data_collator=data_collator,  # Usar o collator personalizado
    )
    
    trainer.train()
    eval_results = trainer.evaluate()
    
    return trainer, eval_results

def create_pseudo_labeled_dataset(unlabeled_tokenized, pseudo_labels):
    """
    Cria um dataset com pseudo-rótulos.
    
    Args:
        unlabeled_tokenized: Tensores tokenizados de dados não rotulados
        pseudo_labels: Lista de pseudo-rótulos gerados pelo modelo
        
    Returns:
        pseudo_labeled_dataset: TensorDataset com dados pseudo-rotulados
    """
    return TensorDataset(
        unlabeled_tokenized["input_ids"],
        unlabeled_tokenized["attention_mask"],
        torch.tensor(pseudo_labels, dtype=torch.long)  # Garantir que os rótulos sejam do tipo long
    )

## 7. Execução do Fluxo de Trabalho de Aprendizado Semi-Supervisionado

In [ ]:
# Parâmetros configuráveis
MAX_SAMPLES = 500  # Limite de amostras para execução rápida (None para usar todo o dataset)
LABELED_RATIO = 0.2  # Proporção de dados rotulados (20%)
MODEL_NAME = "distilbert/distilbert-base-uncased"  # Modelo pré-treinado
OUTPUT_DIR = "./results"  # Diretório para salvar resultados
SEED = 42  # Semente para reprodutibilidade

In [ ]:
# Configurar sementes para reprodutibilidade
set_seed(SEED)

# Determinar dispositivo (GPU ou CPU)
device = get_device()
print(f"Usando dispositivo: {device}")

# Carregar o dataset
print("Carregando dataset...")
dataset = load_symptom_diagnosis_dataset(max_samples=MAX_SAMPLES)
print(f"Dataset carregado com {len(dataset['train'])} amostras de treino")

# Mostrar alguns exemplos
print_dataset_examples(dataset)

In [ ]:
# Preparar os rótulos (diagnósticos)
label_encoder, num_labels = prepare_labels(dataset)
print(f"Número de diagnósticos possíveis: {num_labels}")
print(f"Diagnósticos: {label_encoder.classes_}")

# Inicializar tokenizer e modelo
tokenizer = get_tokenizer(MODEL_NAME)

# Tokenizar o dataset
tokenized_dataset = tokenize_dataset(dataset, tokenizer)

In [ ]:
# Criar divisão semi-supervisionada
labeled_dataset, unlabeled_dataset, unlabeled_indices, labeled_indices, unlabeled_tokenized = create_semi_supervised_split(
    tokenized_dataset, dataset, label_encoder, tokenizer, labeled_ratio=LABELED_RATIO
)

print(f"Usando {len(labeled_indices)} amostras rotuladas e {len(unlabeled_indices)} não-rotuladas")

# Dividir dados rotulados em treino e validação
labeled_train_dataset, labeled_val_dataset = split_train_val(labeled_dataset)

print(f"Conjunto de treino rotulado: {len(labeled_train_dataset)} amostras")
print(f"Conjunto de validação: {len(labeled_val_dataset)} amostras")

In [ ]:
# Inicializar o modelo
model = get_model(MODEL_NAME, num_labels=num_labels)
model.to(device)

# FASE 1: Treinamento supervisionado com dados rotulados
print("\nTreinando modelo inicial com dados rotulados...")
supervised_trainer, eval_results = train_supervised_model(
    model, tokenizer, labeled_train_dataset, labeled_val_dataset, 
    compute_metrics, output_dir=f"{OUTPUT_DIR}/supervised"
)

print(f"\nResultados após treinamento supervisionado:")
print(f"Acurácia: {eval_results['eval_accuracy']:.4f}")
print(f"F1-score: {eval_results['eval_f1']:.4f}")

In [ ]:
# FASE 2: Pseudo-rotulagem
print("\nIniciando fase de pseudo-rotulagem...")

# Criar DataLoader para dados não rotulados
unlabeled_dataloader = DataLoader(
    unlabeled_dataset,
    batch_size=32,
    shuffle=False
)

# Gerar pseudo-rótulos com o modelo treinado
pseudo_labels = generate_pseudo_labels(model, unlabeled_dataloader, device)

# Criar dataset com pseudo-rótulos
pseudo_labeled_dataset = create_pseudo_labeled_dataset(unlabeled_tokenized, pseudo_labels)

In [ ]:
# FASE 3: Treinamento semi-supervisionado
print("\nTreinando modelo com dataset combinado (rotulado + pseudo-rotulado)...")
semi_supervised_trainer, final_eval_results = train_semi_supervised_model(
    model, pseudo_labeled_dataset, labeled_train_dataset, labeled_val_dataset, 
    compute_metrics, output_dir=f"{OUTPUT_DIR}/semi_supervised"
)

print(f"\nResultados após treinamento semi-supervisionado:")
print(f"Acurácia: {final_eval_results['eval_accuracy']:.4f}")
print(f"F1-score: {final_eval_results['eval_f1']:.4f}")

In [ ]:
# Comparar resultados
print_comparison_results(eval_results, final_eval_results)

print("\nDemonstração de aprendizado semi-supervisionado concluída!")

## 8. Conclusão

Neste notebook, demonstramos como implementar o aprendizado semi-supervisionado usando a técnica de pseudo-rotulagem para classificação de textos médicos. O processo envolveu:

1. Treinar um modelo inicial com uma pequena porção de dados rotulados (20%)
2. Usar este modelo para gerar "pseudo-rótulos" para os dados não rotulados (80%)
3. Treinar um novo modelo combinando os dados rotulados originais com os dados pseudo-rotulados
4. Comparar o desempenho dos modelos antes e depois do uso de dados pseudo-rotulados

Os resultados mostram que, mesmo com uma quantidade limitada de dados rotulados, podemos aproveitar dados não rotulados para melhorar o desempenho do modelo através do aprendizado semi-supervisionado.